In [7]:
from helper import SocketListener
from concurrent.futures import ThreadPoolExecutor
from pynput.keyboard import Key, Controller
import time
import random

In [ ]:
data_queue = []
# [Miss, Meh, Ok, Good, Great, Perfect, Passed?, Failed?]
hit_counts = [0] * 8
keyboard = Controller()

def data_handler(data):
  data_queue.append(data)
  
listener = SocketListener(data_handler=data_handler)
  
def process_data_queue():
  while listener.is_listening or listener.is_first_connection:
    if data_queue:
      data = data_queue.pop(0)
      
      data = int.from_bytes(data, byteorder='little')
      if data == 6:
        print('Song passed')
      if data == 7:
        print('Song failed')
      
      hit_counts[data] += 1

# Immediately ends connection after a song ends
def check_connection():
  while True:
    if not listener.is_first_connection and not listener.has_connection:
      listener.stop()
      print(hit_counts)
      break
    
def random_input():
  inputs = ['s', 'd', 'k', 'l']
  song_started = False
  
  while listener.is_listening or listener.is_first_connection:
    if listener.has_connection:
      song_started = True
      inp = random.choice(inputs)
      keyboard.press(inp)
      keyboard.release(inp)
      time.sleep(0.01)
    elif song_started:
      # There will be some extra inputs because of delay after song ends
      # Those inputs will engage the search bar, clearing them
      song_started = False
      for _ in range(10):
        keyboard.press(Key.backspace)
        keyboard.release(Key.backspace)
  
  print('exit')

with ThreadPoolExecutor(max_workers=8) as executor:
    # executor.submit(random_input)
    executor.submit(check_connection)
    executor.submit(process_data_queue)
    executor.submit(listener.start)

Listening on 127.0.0.1:5555
Connection from ('127.0.0.1', 49437)
Song failed
Connection closed.[15, 0, 0, 0, 0, 0, 0, 1]

Socket listener stopped.
